### Filtering Data with SQL
### Objectives
You will be able to:

- Retrieve a subset of records from a table using a WHERE clause
- Filter results using conditional operators such as BETWEEN, IS NULL, and LIKE
- Apply an aggregate function to the result of a filtered query

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data/data.sqlite')

In [3]:
! ls

Database Admin 101 - Lab.ipynb
Database Admin 101.ipynb
Filtering Data with SQL - Lab.ipynb
Filtering, Ordering, and Limiting Data with SQL - Lab.ipynb
Getting Started with SQL - Recap.ipynb
Grouping Data with SQL - Lab.ipynb
Ordering and Limiting Data with SQL.ipynb
SQL Database Data Types.ipynb
connect to SQL Databases.ipynb
connect to SQL Databases2.ipynb
contact_list.pickle
data
data.sqlite
filtering.ipynb
flights.db
grouping data with SQL.ipynb
pets_database.db
school.sqlite
selectData.ipynb
sql_queries.ipynb


In [4]:
#method 1
pd.read_sql("""
SELECT *
  FROM employees;
""", conn).head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


When filtering data using WHERE, you are trying to find rows that match a specific condition. The simplest condition involves checking whether a specific column contains a specific value. In SQLite, this is done using =, which is similar to == in Python:

In [5]:
pd.read_sql(''' 
SELECT * 
FROM employees
WHERE lastName = "Patterson"
''',conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
1,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
2,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


Note that we are selecting all columns (SELECT *) but are no longer selecting all rows. Instead, we are only selecting the 3 rows where the value of lastName is "Patterson".

SQL is essentially doing something like this:

In [6]:
# Selecting all of the records in the database
result = pd.read_sql("SELECT * FROM employees;", conn)
# Create a list to store the records that match the query
employees_named_patterson = []
# Loop over all of the employees
for _, data in result.iterrows():
    # Check if the last name is "Patterson"
    if data["lastName"] == "Patterson":
        # Add to list
        employees_named_patterson.append(data)

# Display the result list as a DataFrame
pd.DataFrame(employees_named_patterson)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


Except SQL is designed specifically to perform these kinds of queries efficiently! **Even if you are pulling data from SQL into Python for further analysis, SELECT * FROM <table>; is very rarely the most efficient approach**. You should be thinking about how to get SQL to do the "heavy lifting" for you in terms of selecting, filtering, and transforming the raw data!

You can also combine WHERE clauses with SELECT statements other than SELECT * in order to filter rows and columns at the same time. For example:

In [7]:
pd.read_sql(''' 
SELECT firstName, lastName, email
FROM employees
WHERE lastName = "Patterson"
''',conn)

,firstName,lastName,email
0,Mary,Patterson,mpatterso@classicmodelcars.com
1,William,Patterson,wpatterson@classicmodelcars.com
2,Steve,Patterson,spatterson@classicmodelcars.com


WHERE clauses are especially powerful when combined with more-complex SELECT statements. Most of the time you will want to use aliases (with AS) in the SELECT statements to make the WHERE clauses more concise and readable.
### Selecting Employees Based on String Conditions
If we wanted to select all employees with 5 letters in their first name, that would look like this:

In [8]:
pd.read_sql(''' 
SELECT *, LENGTH(firstName) AS name_length
FROM employees 
WHERE LENGTH(firstName) = 5
''',conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,name_length
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President,5
1,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,5
2,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,5
3,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep,5
4,1504,Jones,Barry,x102,bjones@classicmodelcars.com,7,1102,Sales Rep,5
5,1612,Marsh,Peter,x102,pmarsh@classicmodelcars.com,6,1088,Sales Rep,5


Or, to select all employees with the first initial of "L", that would look like this:

In [9]:
pd.read_sql(''' 
SELECT *
FROM employees 
WHERE substr(firstName,1,1) = "L"
''',conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
1,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
2,1337,Bondur,Loui,x6493,lbondur@classicmodelcars.com,4,1102,Sales Rep
3,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep


In [10]:
#Teachers code
pd.read_sql(''' 
SELECT *,substr(firstName,1,1) as first_initial
FROM employees
WHERE first_initial = "L";
''',conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,first_initial
0,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep,L
1,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep,L
2,1337,Bondur,Loui,x6493,lbondur@classicmodelcars.com,4,1102,Sales Rep,L
3,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep,L


### Selecting Order Details Based on Price
Below we select all order details where the price each, rounded to the nearest integer, is 30 dollars:

In [11]:
pd.read_sql('''
SELECT *,CAST(round(priceEach) AS INTEGER) AS price_int
FROM orderdetails
WHERE price_int=30
            
''',conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,price_int
0,10104,S24_2840,44,30.41,10,30
1,10173,S24_1937,31,29.87,9,30
2,10184,S24_2840,42,30.06,7,30
3,10280,S24_1937,20,29.87,12,30
4,10332,S24_1937,45,29.87,6,30
5,10367,S24_1937,23,29.54,13,30
6,10380,S24_1937,32,29.87,4,30


In [12]:
pd.read_sql('''
SELECT *
FROM orderdetails
where   CAST(round(priceEach) AS INT) =30         
''',conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10104,S24_2840,44,30.41,10
1,10173,S24_1937,31,29.87,9
2,10184,S24_2840,42,30.06,7
3,10280,S24_1937,20,29.87,12
4,10332,S24_1937,45,29.87,6
5,10367,S24_1937,23,29.54,13
6,10380,S24_1937,32,29.87,4


### Selecting Orders Based on Date
We can use the strftime function to select all orders placed in January of any year:


In [13]:
pd.read_sql(''' 
SELECT * ,strftime('%m',orderDate) as Month
FROM orders
WHERE Month='01'
''',conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,Month
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,01
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,01
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181,01
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121,01
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141,01
5,10208,2004-01-02,2004-01-11,2004-01-04,Shipped,,146,01
6,10209,2004-01-09,2004-01-15,2004-01-12,Shipped,,347,01
7,10210,2004-01-12,2004-01-22,2004-01-20,Shipped,,177,01
8,10211,2004-01-15,2004-01-25,2004-01-18,Shipped,,406,01
9,10212,2004-01-16,2004-01-24,2004-01-18,Shipped,,141,01


We can also check to see if any orders were shipped late (shippedDate after requiredDate, i.e. the number of days late is a positive number):

In [14]:
pd.read_sql('''
SELECT julianday(shippedDate) - julianday(requiredDate) as days_late ,* 
FROM orders
WHERE days_late >0
 ''',conn)

,days_late,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,56.0,10165,2003-10-22,2003-10-31,2003-12-26,Shipped,This order was on hold because customers's cre...,148


### Conditional Operators in SQL
In all of the above queries, we used the = operator to check if we had an exact match for a given value. However, what if you wanted to select the order details where the price was at least 30 dollars? Or all of the orders that don't currently have a shipped date?

We'll need some more advanced conditional operators for that.

Some important ones to know are:

- != ("not equal to")
Similar to not combined with == in Python
- > ("greater than")
Similar to > in Python
- >= ("greater than or equal to")
Similar to >= in Python
- < ("less than")
Similar to < in Python
- <= ("less than or equal to")
Similar to <= in Python
- AND
Similar to and in Python
- OR
Similar to or in Python
- BETWEEN
Similar to placing a value between two values with <= and and in Python, e.g. (2 <= x) and (x <= 5)
- IN
Similar to in in Python
- LIKE
Uses wildcards to find similar strings. No direct equivalent in Python, but similar to some Bash terminal commands.
### Cats Data
For this section as the queries get more advanced we'll be using a simpler database called pets_database.db containing a table called cats.

In [15]:
!ls

Database Admin 101 - Lab.ipynb
Database Admin 101.ipynb
Filtering Data with SQL - Lab.ipynb
Filtering, Ordering, and Limiting Data with SQL - Lab.ipynb
Getting Started with SQL - Recap.ipynb
Grouping Data with SQL - Lab.ipynb
Ordering and Limiting Data with SQL.ipynb
SQL Database Data Types.ipynb
connect to SQL Databases.ipynb
connect to SQL Databases2.ipynb
contact_list.pickle
data
data.sqlite
filtering.ipynb
flights.db
grouping data with SQL.ipynb
pets_database.db
school.sqlite
selectData.ipynb
sql_queries.ipynb


Below we make a new database connection and read all of the data from this table:

In [16]:
conn = sqlite3.connect('data\pets_database.db')
pd.read_sql('select * from cats;',conn)

,id,name,age,breed,owner_id
0,1,Maru,3.0,Scottish Fold,1.0
1,2,Hana,1.0,Tabby,1.0
2,3,Lil' Bub,5.0,American Shorthair,NaN
3,4,Moe,10.0,Tabby,NaN
4,5,Patches,2.0,Calico,NaN
5,6,None,NaN,Tabby,NaN


### WHERE Code-Along
In this exercise, you'll walk through executing a handful of common and handy SQL queries that use WHERE with conditional operators. We'll start by giving you an example of what this type of query looks like, then have you type a query specifically related to the cats table.

### WHERE with >=
For the =, !=, <, <=, >, and >= operators, the query looks like:

`SELECT column(s)`

  `FROM table_name`
  
 `WHERE column_name operator value;`
 
Note: The example above is not valid SQL, it is a template for how the queries are constructed

Type this SQL query between the quotes below to select all cats who are at least 5 years old:

In [17]:
pd.read_sql('''
SELECT *
FROM cats
WHERE age >=5         
 ''',conn)

,id,name,age,breed,owner_id
0,3,Lil' Bub,5,American Shorthair,None
1,4,Moe,10,Tabby,None


### WHERE with BETWEEN
If you wanted to select all rows with values in a range, you could do this by combining the <= and AND operators. However, since this is such a common task in SQL, there is a shorter and more efficient command specifically for this purpose, called BETWEEN.

A typical query with BETWEEN looks like:

`SELECT column_name(s)`

  `FROM table_name`

` WHERE column_name BETWEEN value1 AND value2;`

Note that `BETWEEN` is an inclusive range, so the returned values can match the boundary values (not like range() in Python)

Let's say you need to select the names of all of the cats whose age is between 1 and 3. Type this SQL query between the quotes below to select all cats who are in this age range:

In [18]:
pd.read_sql(''' 
SELECT * 
FROM cats
WHERE age BETWEEN 1 AND 3
''',conn)

,id,name,age,breed,owner_id
0,1,Maru,3,Scottish Fold,1.0
1,2,Hana,1,Tabby,1.0
2,5,Patches,2,Calico,NaN


### WHERE Column Is Not NULL
NULL in SQL represents missing data. It is similar to None in Python or NaN in NumPy or pandas. However, we use the IS operator to check if something is NULL, not the = operator (or IS NOT instead of !=).

To check if a value is NULL (or not), the query looks like:

`SELECT column(s)`

  `FROM table_name`
  
 `WHERE column_name IS (NOT) NULL;`
 
You might have noticed when we selected all rows of cats, some owner IDs were NaN, then in the above query they are None instead. This is a subtle difference where Python/pandas is converting SQL NULL values to NaN when there are numbers in other rows, and converting to None when all of the returned values are NULL. This is a subtle difference that you don't need to memorize; it is just highlighted to demonstrate that the operators we use in SQL are similar to Python operators, but not quite the same.

If we want to select all cats that don't currently belong to an owner, we want to select all cats where the owner_id is NULL.

Type this SQL query between the quotes below to select all cats that don't currently belong to an owner:

In [19]:
pd.read_sql(''' 
SELECT * 
FROM cats
WHERE owner_id IS NULL
''',conn)

,id,name,age,breed,owner_id
0,3,Lil' Bub,5.0,American Shorthair,None
1,4,Moe,10.0,Tabby,None
2,5,Patches,2.0,Calico,None
3,6,None,NaN,Tabby,None


### WHERE with LIKE
The LIKE operator is very helpful for writing SQL queries with messy data. It uses wildcards to specify which parts of the string query need to be an exact match and which parts can be variable.

When using LIKE, a query looks like:

`SELECT column(s)`

  `FROM table_name`

 `WHERE column_name LIKE 'string_with_wildcards';`
 
The most common wildcard you'll see is %. This is similar to the * wildcard in Bash or regex: it means zero or more characters with any value can be in that position.

So for example, if we want all cats with names that start with "M", we could use a query containing M%. This means that we're looking for matches that start with one character "M" (or "m", since this is a case-insensitive query in SQLite) and then zero or more characters that can have any value.

Type this SQL query between the quotes below to select all cats with names that start with "M" (or "m"):

In [20]:
pd.read_sql(''' 
SELECT *
FROM cats
WHERE name like 'm%'
''',conn)

,id,name,age,breed,owner_id
0,1,Maru,3,Scottish Fold,1.0
1,4,Moe,10,Tabby,NaN


Unlike in Python where:

There should be one-- and preferably only one --obvious way to do it. (Zen of Python)

there will often be multiple valid approaches to writing the same SQL query. Sometimes one will be more efficient than the other, and sometimes the only difference will be a matter of preference.

The other wildcard used for comparing strings is _, which means exactly one character, with any value.

For example, if we wanted to select all cats with four-letter names where the second letter was "a", we could use _a__.

Type this SQL query between the quotes below to select all cats with names where the second letter is "a" and the name is four letters long:

In [21]:
pd.read_sql(''' 
SELECT * 
FROM cats
WHERE name like '_a__'
''',conn)

,id,name,age,breed,owner_id
0,1,Maru,3,Scottish Fold,1
1,2,Hana,1,Tabby,1


Again, we could have done this using length and substr, although it would be much less concise:

In [22]:
pd.read_sql(''' 
SELECT * 
FROM cats
WHERE length(name)=4  and substr(name,2,1)='a'
''',conn)

,id,name,age,breed,owner_id
0,1,Maru,3,Scottish Fold,1
1,2,Hana,1,Tabby,1


These examples are a bit silly, but you can imagine how this technique would help to write queries between multiple datasets where the names don't quite match exactly! You can combine % and _ in your string to narrow and expand your query results as needed.

### SELECT with COUNT
Now, let's talk about the SQL aggregate function COUNT.

**SQL aggregate functions** are SQL statements that can get the average of a column's values, retrieve the minimum and maximum values from a column, sum values in a column, or count a number of records that meet certain conditions. You can learn more about these SQL aggregators here and here.

For now, we'll just focus on COUNT, which counts the number of records that meet a certain condition. Here's a standard SQL query using COUNT:

`SELECT COUNT(column_name)`

  `FROM table_name`

 `WHERE conditional_statement;`
 
Let's try it out and count the number of cats who have an owner_id of 1. Type this SQL query between the quotes below:

In [33]:
pd.read_sql(''' 
SELECT count(owner_id)
FROM cats
WHERE owner_id=1
''',conn)

,count(owner_id)
0,2


### Note on SELECT
We are now familiar with this syntax:

`SELECT name`
  `FROM cats;`
  
However, you may not know that this can be written like this as well:

`SELECT cats.name`
  `FROM cats;`
  
Both return the same data.

SQLite allows us to explicitly state the tableName.columnName you want to select. This is particularly useful when you want data from two different tables.

Imagine you have another table called dogs with a column containing all of the dog names:

In [24]:
pd.read_sql(''' 
SELECT name
FROM cats
''',conn)

,name
0,Maru
1,Hana
2,Lil' Bub
3,Moe
4,Patches
5,None


In [25]:
pd.read_sql(''' 
SELECT cats.name
FROM cats
''',conn)

,name
0,Maru
1,Hana
2,Lil' Bub
3,Moe
4,Patches
5,None


In [26]:
conn.execute(''' 
CREATE TABLE if not exists dogs(
            id INTEGER primary key,
            name TEXT
            )
''')

In [27]:
conn.execute(''' 
INSERT INTO dogs (name)
VALUES
('Clifford'),
('Jack'),
('Lily')
''')
conn.commit()

In [28]:
pd.read_sql(''' 
select * from dogs
''',conn)

,id,name
0,1,Clifford
1,2,Jack
2,3,Lily
3,4,Clifford
4,5,Jack
5,6,Lily
6,7,Clifford
7,8,Jack
8,9,Lily


If you want to get the names of all the dogs and cats, you can no longer run a query with just the column name. SELECT name FROM cats,dogs; will return Error: ambiguous column name: name.

In [35]:
pd.read_sql(''' 
select name from cats,dogs
''',conn)

DatabaseError: Execution failed on sql ' 
select name from cats,dogs
': ambiguous column name: name

In [74]:
pd.read_sql(''' 
select cats.name,dogs.name 
from cats,dogs;
''',conn)

,name,name
0,Maru,Clifford
1,Maru,Jack
2,Maru,Lily
3,Maru,Clifford
4,Maru,Jack
5,Maru,Lily
6,Hana,Clifford
7,Hana,Jack
8,Hana,Lily
9,Hana,Clifford


In [75]:
conn.close()

### Summary
In this lesson, you saw how to filter the resulting rows of a SQL query using a WHERE clause that checked whether a given column was equal to a specific value. You also got a basic introduction to aggregate functions by seeing an example of COUNT, and dove deeper into some conditional operators including BETWEEN and LIKE.